In [ ]:
from common import *
data = load_json('3.json', True)

def hack_fightprops(l: list):
    output = {}
    for prop in l:
        if 'propType' in prop and 'propValue' in prop:
            output[str(prop['propType'])] = prop['propValue']
    return 'fightProps', output

data_flat = sorted([flatten_json(
    d, flatten_keys=['gadget', 'motionInfo', 'gatherGadget'], stripped_keys=['entityId'], flatten_xyz=True, flatten_lists=False, key_hacks={'fightPropList':hack_fightprops}) for d in data],
    key=lambda x: (len(x), x.get('gadgetId', 0)))
df = pd.DataFrame(data_flat)
#df1 = df.loc[:,df.nunique(dropna=False)!=1].sort_values(['itemId', 'groupId', 'configId', 'gadgetId'])
df1 = df.loc[:,df.nunique(dropna=False)!=1].sort_values(['pos', 'configId', 'groupId'])

In [ ]:
def make_gatherable_templates(df: pd.DataFrame) -> list:
    def make_template(df, name=''):
        common_values = df.loc[:, df.nunique(dropna=False) == 1]
        varying_values = df.loc[:, df.nunique(dropna=False) > 1]
        output = {'TemplateName': name} if name else {}
        for k,v in common_values.iloc[0].iteritems():
            output[k] = v
        # TODO: something more stringent
        if name:
            # print(f'Recursing for name {name}')
            vv = varying_values.copy()
            vv_gid = vv.get('groupId', None)
            if vv_gid is not None:
                group_counts = vv_gid.value_counts()
                one_ofs = group_counts[group_counts == 1].index
                multiple_ofs = group_counts[group_counts > 1].index
                output['zChildren'] = []
                for group_id in multiple_ofs:
                    output['zChildren'].append(make_template(vv[vv_gid == group_id]))
                if len(one_ofs) > 0:
                    output['zChildren'] += vv[vv_gid.isin(one_ofs)].to_dict('records')
            else:
                output['zChildren'] = varying_values.to_dict('records')    
        else:
            output['zChildren'] = varying_values.to_dict('records')
        return output

    gatherable_counts = df['itemId'].value_counts()
    one_ofs = gatherable_counts[gatherable_counts == 1].index
    multiple_ofs = gatherable_counts[gatherable_counts > 1].index
    outputs = []
    for gatherable_id in multiple_ofs:
        name = handbook_items.get(int(gatherable_id), gatherable_id)
        outputs.append(make_template(df[df['itemId'] == gatherable_id], name))
    outputs.append(make_template(df[df['itemId'].isin(one_ofs)], 'one_ofs'))
    outputs.append(make_template(df[df['itemId'].isnull()], 'NaNs'))
    return outputs

t = make_gatherable_templates(df1)

In [ ]:
ts = encode_json(t, -2)
with open_write('3.pandas.json', True) as f:
    f.write(ts)

In [ ]:
compact_keys = {'gadgetId': 0, 'configId': 0, 'groupId': 0, 'itemId': 0, 'pos': (0,0,0), 'rot': (0,0,0), 'isEnableInteract': 0, 'authorityPeerId': 0}
def compact_record(d: dict):
    output = {}
    output['_'] = tuple(d.get(k, v) for k,v in compact_keys.items())
    for k, v in d.items():
        if k not in compact_keys:
            output[k] = v
    return output
df1d = df1.to_dict(orient='records')
df1dc = [compact_record(d) for d in df1d]
sorted_3json = encode_json(df1dc, -2, maxline=90000, nan_to_null=True)
with open_write('3.records.json', True) as f:
    f.write(sorted_3json)

In [ ]:
textmaps = load_textmaps()
OptionExcelConfigData = pd.read_json(GC_PATH + 'resources/ExcelBinOutput/OptionExcelConfigData.json')
OptionExcelConfigData['contentTextMapHash'] = OptionExcelConfigData['contentTextMapHash'].map(lambda x: textmaps['EN'].get(str(x)))
OptionExcelConfigData.to_excel('OptionExcelConfigDataNames.xlsx', index=False)

In [ ]:
GatherExcelConfigData = pd.read_json('resources/ExcelBinOutput/GatherExcelConfigData.json')
GatherExcelConfigData['itemId'] = GatherExcelConfigData['itemId'].map(handbook_items.get)

In [ ]:
keys = textmaps['EN'].keys()
reverse_mappings = {}
for key in keys:
    st = tuple(textmaps[l].get(key, '') for l in ('EN', 'JP', 'CHS', 'CHT', 'RU'))
    reverse_mappings[st] = reverse_mappings.get(st, []) + [key]

reverse_mappings_sorted = sorted(reverse_mappings.items(), key=lambda x: len(x[1]), reverse=True)
for st, hashes in reverse_mappings_sorted[:2500]:
    # print(f'{st}: {len(hashes)} copies: {", ".join(hashes)}')
    print(f'{len(hashes)} copies of \t{st}')

In [ ]:
item_ids = set(df.loc[:,'itemId'])
gadget_ids = set(df.loc[:,'gadgetId'])
def item_name(id: int) -> str:
    return handbook_items.get(id, str(id))
i2g = {item_name(id): sorted(set(df[df['itemId']==id]['gadgetId'])) for id in item_ids}
g2i = {id: sorted({item_name(i) for i in set(df[df['gadgetId']==id]['itemId'])}) for id in gadget_ids}

print('itemId : gadgetIds')
print('\n'.join(f'{k}: {v}' for k,v in sorted(i2g.items(), key=lambda x: (-len(x[1]), x[0]))))
print('gadgetId : itemIds')
print('\n'.join(f'{k}: {v}' for k,v in sorted(g2i.items(), key=lambda x: (-len(x[1]), x[1][0]))))

In [ ]:
mint_ids = [70520009, 70520024]
positions = [df1[df1['gadgetId']==id].loc[:,'pos'] for id in mint_ids]
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [10, 10]
plt.axes().set_aspect('equal')
for pos in positions:
    plt.scatter([-p[2] for p in pos], [p[0] for p in pos], alpha=0.5)
plt.show()

In [ ]:
spawns = load_json('src/main/resources/defaults/data/GadgetSpawns.json')
spawns3 = [spawn for spawn in spawns if spawn['sceneId']==3]
def pos2xy(pos: dict) -> list:  # Rotate to our screenspace
    return [-pos.get("z", 0), pos.get("x", 0)]
def spawns2array(spawns: list) -> np.ndarray:
    # return np.array([pos2xy(s['pos']) for s in spawns if 'pos' in s])
    return [pos2xy(s['pos']) for s in spawns if 'pos' in s]
# centroids = {s['groupId']:pos2xy(s['pos']) for s in spawns3 if 'groupId' in s}
# spawn_points = {s['groupId']:spawns2array(s['spawns']) for s in spawns3 if 'groupId' in s}
# gid_to_centroid_and_points = {s['groupId']:(pos2xy(s['pos']), spawns2array(s['spawns'])) for s in spawns3 if 'groupId' in s}
data = []
gids = []
for s in spawns3:
    if 'groupId' not in s:
        continue
    gid = s['groupId']
    centroid = pos2xy(s['pos'])
    for p in spawns2array(s['spawns']):
        data.append([p, centroid])
        gids.append(gid)
spawn_points = np.array(data)
colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf']
cs = [colors[i%len(colors)] for i in gids]


import matplotlib.pyplot as plt
import matplotlib.collections as mc
from matplotlib.ticker import (AutoMinorLocator, MultipleLocator)
%matplotlib widget
# %matplotlib inline
plt.rcParams['figure.figsize'] = [20, 20]
# plt.rcParams['figure.dpi'] = 300
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
lc = mc.LineCollection(spawn_points, linewidths=2, alpha=0.05, colors=cs)
ax.set_aspect('equal')
ax.add_collection(lc, autolim=False)
ax.scatter(spawn_points[:,0,0], spawn_points[:,0,1], c=cs, alpha=0.5)

# Change major ticks to show every 20.
ax.xaxis.set_major_locator(MultipleLocator(128))
ax.yaxis.set_major_locator(MultipleLocator(128))

# Change minor ticks to show every 5. (20/4 = 5)
ax.xaxis.set_minor_locator(AutoMinorLocator(4))
ax.yaxis.set_minor_locator(AutoMinorLocator(4))
ax.grid()
# plt.plot()